In [70]:
%pwd

'c:\\Users\\HP'

In [72]:
os.chdir(r'C:\Users\HP\My End to End Modular Coding Projects\Wine Quality Prediction\End-to-End-Wine-Quality-Prediction')

In [73]:
%pwd

'C:\\Users\\HP\\My End to End Modular Coding Projects\\Wine Quality Prediction\\End-to-End-Wine-Quality-Prediction'

In [74]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [75]:
from WineQuality.constants import *
from WineQuality.utils.common import read_yaml, create_directories

In [76]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [77]:
import urllib.request as request
import zipfile
from WineQuality import logger
from WineQuality.utils.common import get_size

In [78]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded ! with following info {headers}")
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [79]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-09 13:38:18,733: INFO: common: 18: yaml file: config\config.yaml loaded successfully]
[2024-11-09 13:38:18,733: INFO: common: 18: yaml file: params.yaml loaded successfully]
[2024-11-09 13:38:18,741: INFO: common: 18: yaml file: schema.yaml loaded successfully]
[2024-11-09 13:38:18,741: INFO: common: 30: Created Directory at artifacts]
[2024-11-09 13:38:18,749: INFO: common: 30: Created Directory at artifacts/data_ingestion]
[2024-11-09 13:38:21,479: INFO: 1665351981: 11: artifacts/data_ingestion/MyData.zip downloaded ! with following info Connection: close
Content-Length: 21966
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8782c21e95cf0161d880443982ddd3be6a05d7b721e90426da36c0f7c50df613"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BCDD:2523EE:3AE20:493F7:672F1876
Accept-Rang